In [1]:
import re
from qiskit import QuantumCircuit,transpile
import qiskit
import math

In [2]:
import os
def load(path,name):
    with open(path+name,"r") as f:
        lines = f.readlines()
    result=[]
    for line in lines:
        result.append(line.strip())
    return result
def strip(lines):
    results=[]
    for line in lines:
        line = line.strip()
        if "measure" in line:
            continue
        elif "barrier" in line:
            continue
        elif "reset" in line:
            continue
        else:
            results.append(line)
    return results    
    
def transpile_qiskit(lines):
    qasm_str="\n".join(lines)
    circ=QuantumCircuit.from_qasm_str(qasm_str)
    transpiled_circ=transpile(circ,basis_gates=['rz', 'cx','h','x',"ccx"],optimization_level=0)
    qasm_str=qiskit.qasm2.dumps(transpiled_circ)
    return qasm_str.split("\n")
def transpile_to_IBM(lines):
    qasm_str="\n".join(lines)
    circ=QuantumCircuit.from_qasm_str(qasm_str)
    transpiled_circ=transpile(circ,basis_gates=['u1', 'u2', 'u3', 'cx'],optimization_level=0)
    qasm_str=qiskit.qasm2.dumps(transpiled_circ)
    return qasm_str.split("\n")
def transpile_to_cliff(lines):
    qasm_str="\n".join(lines)
    circ=QuantumCircuit.from_qasm_str(qasm_str)
    transpiled_circ=transpile(circ,basis_gates=['cx','h','x',"z","t","tdg","s"],optimization_level=0)
    qasm_str=qiskit.qasm2.dumps(transpiled_circ)
    return qasm_str.split("\n")
def save(lines,path,name):
    os.makedirs(path,exist_ok=True)
    new_lines=[]
    for line in lines:
        new_lines.append(line.strip())
    qasm_str="\n".join(lines)
    with open(path+name,"w") as f:
        f.write(qasm_str)
def ccx_to_ccz(lines):
    results=[]
    
    pattern = r'ccx\s+(\w+\[\d+\]),(\w+\[\d+\]),(\w+\[\d+\]);'
    
    def replacement(match):
        q1, q2, q3 = match.groups()
        return f"h {q3};\nccz {q1}, {q2}, {q3};\nh {q3};"
    
    for line in lines:
        if "ccx" in line:
            line = re.sub(pattern, replacement, line)
        results.append(line)

    return results

def printing(names,dir_path):
    for name in names:
        path=dir_path+name+".transpiled"
        print("\""+path+"\",")
def parse_pi(lines):
    results=[]
    for line in lines:
        result=line
        if "u1" in line:
            pattern = r'u1\((.*?)\) q\[(\d+)\];'
            
            def replace_function(match):
                expression = match.group(1)
                qubit = match.group(2)
                evaluated_value = eval(expression, {"__builtins__": None}, {"pi": math.pi})
                return f'u1({evaluated_value}) q[{qubit}];'
            
            result = re.sub(pattern, replace_function, line)
        
        elif "u2" in line:
            pattern = r'u2\((.*?),(.*?)\) q\[(\d+)\];'
            
            def replace_function(match):
                expression1 = match.group(1)
                expression2 = match.group(2)
                qubit = match.group(3)
                evaluated_value1 = eval(expression1, {"__builtins__": None}, {"pi": math.pi})
                evaluated_value2 = eval(expression2, {"__builtins__": None}, {"pi": math.pi})
                return f'u2({evaluated_value1},{evaluated_value2}) q[{qubit}];'
            
            result = re.sub(pattern, replace_function, line)
        
        elif "u3" in line:
            pattern = r'u3\((.*?),(.*?),(.*?)\) q\[(\d+)\];'
            
            def replace_function(match):
                expression1 = match.group(1)
                expression2 = match.group(2)
                expression3 = match.group(3)
                qubit = match.group(4)
                evaluated_value1 = eval(expression1, {"__builtins__": None}, {"pi": math.pi})
                evaluated_value2 = eval(expression2, {"__builtins__": None}, {"pi": math.pi})
                evaluated_value3 = eval(expression3, {"__builtins__": None}, {"pi": math.pi})
                return f'u3({evaluated_value1},{evaluated_value2},{evaluated_value3}) q[{qubit}];'
            
            result = re.sub(pattern, replace_function, line)
        results.append(result)
    return results


In [3]:
names=["binary_welded_tree_n25","binary_welded_tree_n29"]


path="../benchmarks/nwqbench/NWQ_Bench/qasm/"
for name in names:
    dir_path=path
    dir_path_transpiled=path+"transpiled/"+"nwq_"+name+"/"
    file_name=name+".qasm"
    file_name_transpiled="nwq_"+name+".qasm.bak"
    lines=load(dir_path,file_name)
    lines=strip(lines)
    save(lines,dir_path_transpiled,file_name_transpiled)

In [6]:
names=["binary_welded_tree_n25","binary_welded_tree_n29"]
path="../benchmarks/nwqbench/NWQ_Bench/qasm/transpiled/"
for name in names:
    dir_path=path+"nwq_"+name+"/"
    file_name="nwq_"+name+".qasm.bak"
    file_name_transpiled="nwq_"+name+".qasm"
    lines=load(dir_path,file_name)
    lines=transpile_qiskit(lines)
    lines=ccx_to_ccz(lines)
    save(lines,dir_path,file_name_transpiled)

In [47]:
# for name in names:

#     dir_path=path+"nwq_"+name+"/"
#     file_name="nwq_"+name+".qasm.bak"
#     file_name_transpiled="nwq_"+name+".clifft.qasm"
#     lines=load(dir_path,file_name)
#     lines=transpile_to_cliff(lines)
#     save(lines,dir_path,file_name_transpiled)


In [10]:
for name in names:
    print(name)
    dir_path=path+"nwq_"+name+"/"
    file_name="nwq_"+name+".qasm.bak"
    file_name_transpiled="nwq_"+name+".ibm.qasm"
    lines=load(dir_path,file_name)
    lines=transpile_to_IBM(lines)
    lines=parse_pi(lines)
    save(lines,dir_path,file_name_transpiled)

square_root_n48
statevector_n5
